In [4]:
import gradio as gr

def hello(name):
    return f'{name}님 안녕!', ""

with gr.Blocks() as demo:
    name_textbox = gr.Textbox(label="이름", placeholder = "이름을 입력하세요")
    output_textbox = gr.Textbox(label="출력")
    submit_button = gr.Button("전송")

    submit_button.click(fn=hello, inputs=[name_textbox], outputs=[output_textbox, name_textbox])

demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [5]:
from dotenv import load_dotenv
import os
import gradio as gr
import requests

# 환경변수 로드
load_dotenv()

API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
ENDPOINT = os.getenv("AZURE_CHAT_ENDPOINT")  # ← 이렇게 직접 불러옴

def request_gpt(user_text, temperature=0.7, top_p=0.95, max_tokens=800):
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY
    }
    payload = {
        "messages": [
            {"role": "system", "content": "너는 과일 전문가야. 100자 이내로 귀엽게 설명"},
            {"role": "user", "content": user_text}
        ],
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens
    }

    response = requests.post(ENDPOINT, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"요청 실패: {response.status_code} - {response.text}"

def click_send(input_message, chat_history):
    bot_message = request_gpt(input_message)
    chat_history.append((input_message, bot_message))
    return '', chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    with gr.Row():
        input_textbox = gr.Textbox()
        send_button = gr.Button("Send")
    send_button.click(fn=click_send, inputs=[input_textbox, chatbot], outputs=[input_textbox, chatbot])
    demo.launch()


/var/folders/70/l808w3hj0vq59thqmp6735bc0000gn/T/ipykernel_61806/1278240273.py:39: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
